In [ ]:
'''13/02/2021: Desafio Técnico - Cientista de Dados - A3DATA
               Análise exploratória dos dados de ocorrências aeronáuticas da aviação civil brasileira nos últimos 10 anos.
               Obtidos pela CENIPA - Ocorrências Aeronáuticas na Aviação Civil Brasileira'''

# Importando os pacotes 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from IPython.display import Image
Image('modelo_dados.png')

In [ ]:
# Importando as bases de dados
db_ocor = pd.read_csv("ocorrencia.csv", encoding = 'unicode_escape', sep = ';')
db_ocor_tp = pd.read_csv("ocorrencia_tipo.csv", encoding = 'unicode_escape', sep = ';')
db_aero = pd.read_csv("aeronave.csv", encoding = 'unicode_escape', sep = ';')
db_contr = pd.read_csv("fator_contribuinte.csv", encoding = 'unicode_escape', sep = ';')
db_reco = pd.read_csv("recomendacao.csv", encoding = 'unicode_escape', sep = ';')

In [ ]:
print('Variáveis da base Ocorrência:\n',db_ocor.info())
print('\n\nVariáveis da base Tipo de Ocorrência:\n',db_ocor_tp.info())
print('\n\nVariáveis da base Aeronave:\n',db_aero.info())
print('\n\nVariáveis da base Fator Contribuinte:\n',db_contr.info())
print('\n\nVariáveis da base Recomendação:\n',db_reco.info())

# Análise Geral da Base Ocorrências

In [ ]:
db_ocor.head()

In [ ]:
datas = db_ocor["ocorrencia_dia"].astype("datetime64")

plt.subplot(2,1,1)
datas.groupby([datas.dt.year, datas.dt.month]).count().plot(kind="line", figsize=(8,8), color = 'gray')
plt.xlabel("Ano, mês das ocorrências")
plt.ylabel("Frequência")
plt.title("Distribuição das Ocorrências em relação Mês e Ano")
plt.show()

plt.subplot(2,1,2)
((datas.groupby(datas.dt.year).count().cumsum()/datas.groupby(datas.dt.year).count().sum())*100).plot(kind="bar", figsize=(8,8), color = 'gray')
plt.xlabel("Ano, mês das ocorrências")
plt.ylabel("Frequência  Acumulada")
plt.title("Porcentagem das Ocorrências em relação ao Ano")
plt.show()

In [ ]:
# Distribuição de Ocorrências por Estado e Cidade
x = db_ocor.ocorrencia_cidade.value_counts().index
y = db_ocor.ocorrencia_cidade.value_counts().values
z = (y.cumsum()/y.sum())*100
xn = len(x)

print('Entre 2010 e 2019 houveram ocorrências em %.f cidades.'%(xn))

plt.figure(1,figsize=(8,8))
plt.bar(range(len(x)), z, color = 'gray')
plt.plot([0,246],[80,80], color = 'b')
plt.plot([246,246],[0,80], color = 'b')
plt.ylabel("Porcentagem de ocorrência")
plt.xlabel("Quantidade de Cidades")
plt.title("Porcentagem acumulada das Ocorrências")
plt.show()

print('Tem-se que {0:.2f}% das cidades são responsáveis por {1:.2f}% das ocorrências.'.format((246/1045)*100,(z[246])))

nome = x[:246]
qtd = y[:246]
uf = []

for i in nome:
    ind = db_ocor.ocorrencia_cidade[db_ocor.ocorrencia_cidade==i].index[0]
    uf.append(db_ocor.ocorrencia_uf[ind])

uf = pd.Series(uf, name='Estados')
qtd2=pd.DataFrame(index=range(len(uf.value_counts().index)), columns=['estado','freq'])
s=0
for j in uf.value_counts().index:
    d=0
    for i in range(len(uf)):
        if uf[i]==j:
            d+=qtd[i]
    qtd2.estado[s]=j
    qtd2.freq[s]=d
    s+=1
    
qtd2.sort_values('freq',inplace=True)

a = (db_ocor.ocorrencia_uf.value_counts().values[:-1].sum()/len(db_ocor.ocorrencia_uf))*100
b = (db_ocor.ocorrencia_uf.value_counts().values[:4].sum()/db_ocor.ocorrencia_uf.value_counts().values[:-1].sum())*100
    
plt.figure(2,figsize=(8,8))
plt.barh(db_ocor.ocorrencia_uf.value_counts().index,db_ocor.ocorrencia_uf.value_counts().values, color = 'gray')
plt.ylabel("Estado da ocorrência")
plt.xlabel("Número de Ocorrências")
plt.title("Distribuição de Ocorrências por Estado")
plt.show()

print('Das ocorrências realizadas, {0:.2f}% tiveram estados identificadas. E desses, {1:.2f}% representam os 4 maiores - SP, MG, RJ e PR.'.format(a,b))

plt.figure(3,figsize=(8,8))
plt.barh(qtd2.estado, qtd2.freq, color = 'gray')
plt.ylabel("Estado da ocorrência")
plt.xlabel("Quantidade de Ocorrências")
plt.title("Distribuição por Estado de {0:.2f}% das cidades de maiores frequências".format((246/1045)*100))
plt.show()


In [ ]:
print('No grupo das 246 cidades:')
print(qtd2.estado[-5:])
print(((qtd2.freq/qtd2.freq.sum())*100)[-5:])

In [ ]:
day = []

for i in nome:
    ind = db_ocor.ocorrencia_cidade[db_ocor.ocorrencia_cidade==i].index
    for j in ind:
        day.append(datas[j])

day = pd.Series(day, name='Dias')

day.groupby([day.dt.year, day.dt.month]).count().plot(kind="line", figsize=(8,8), color = 'gray')

plt.subplot(2,1,1)
day.groupby([day.dt.year, day.dt.month]).count().plot(kind="line", figsize=(8,8), color = 'gray')
plt.xlabel("Ano, mês das ocorrências")
plt.ylabel("Frequência")
plt.title("Distribuição das Ocorrências, de {0:.2f}% das cidades de maiores frequências, em relação Mês e Ano".format((246/1045)*100))
plt.show()

plt.subplot(2,1,2)
((day.groupby(day.dt.year).count().cumsum()/day.groupby(day.dt.year).count().sum())*100).plot(kind="bar", figsize=(8,8), color = 'gray')
plt.xlabel("Ano, mês das ocorrências")
plt.ylabel("Frequência  Acumulada")
plt.title("Porcentagem das Ocorrências, de {0:.2f}% das cidades de maiores frequências, em relação ao Ano".format((246/1045)*100))
plt.show()

# Análise Geral da Base Ocorrências_Tipo

In [ ]:
db_ocor_tp.head()

In [ ]:
print(db_ocor_tp.ocorrencia_tipo.value_counts())
print(db_ocor_tp.ocorrencia_tipo_categoria.value_counts())
print(db_ocor_tp.taxonomia_tipo_icao.value_counts())

In [ ]:
# Distribuição de Ocorrências por Tipo de falha
x = db_ocor_tp.ocorrencia_tipo.value_counts().index
y = db_ocor_tp.ocorrencia_tipo.value_counts().values
z = (y.cumsum()/y.sum())*100
xn = len(x)

print('Entre 2010 e 2019 houveram %.f tipos diferentes de falhas.'%(xn))

plt.figure(1,figsize=(8,8))
plt.bar(range(len(x)), z, color = 'gray')
plt.plot([0,15],[80,80], color = 'b')
plt.plot([15,15],[0,80], color = 'b')
plt.ylabel("Porcentagem de ocorrência")
plt.xlabel("Tipos de Falha")
plt.title("Porcentagem acumulada das Ocorrências")
plt.show()

print('Tem-se que {0:.2f}% das falhas, ou seja, 15 tipos diferentes são responsáveis por {1:.2f}% das ocorrências.'.format((15/77)*100,(z[15])))


In [ ]:
# Buscando os códigos das ocorrências desses 15 tipos de falhas mais frequentes
falha = x[:15]
cod_falha = []

for i in falha:
    ind = db_ocor_tp.ocorrencia_tipo[db_ocor_tp.ocorrencia_tipo==i].index
    for j in ind:
        cod_falha.append(db_ocor_tp.codigo_ocorrencia1[j])
        
cod_falha = pd.Series(cod_falha, name='Codigo')

# Buscando os códigos das ocorrências das 246 cidades mais frequentes
cod_ocor = []

for i in nome:
    ind = db_ocor.ocorrencia_cidade[db_ocor.ocorrencia_cidade==i].index
    for j in ind:
        cod_ocor.append(db_ocor.codigo_ocorrencia1[j])
        
cod_ocor = pd.Series(cod_ocor, name='Codigo')

# Comparando os códigos
igual=[]
for i in cod_falha:
    for j in cod_ocor:
        if i==j:
            igual.append(i)
            
print('Comparando as ocorrências das 246 cidades mais frequentes com os 15 tipos de falhas, tem-se que {0:.2f}% dessas são iguais.'.format((len(igual)/len(cod_falha))*100))
#80,24%

In [ ]:
# Mostrar as ocorrências iguais pela localidade (latitude X longitude) e tipo de falha

igual = pd.Series(igual, name='Codigo')
db_igual = pd.DataFrame(index = range(len(igual)), columns = ['Latitude', 'Longitude', 'Falha'])
db_igual.insert(loc = 0, column = 'Codigo', value = igual.values)
for i in igual:
    a = np.where(db_ocor.codigo_ocorrencia1 == i)
    b = np.where(db_igual.Codigo == i)
    c = np.where(db_ocor_tp.codigo_ocorrencia1 == i)
    db_igual.Latitude.loc[b] = db_ocor.ocorrencia_latitude.loc[a].values
    db_igual.Longitude.loc[b] = db_ocor.ocorrencia_longitude.loc[a].values
    db_igual.Falha.loc[b] = str(db_ocor_tp.ocorrencia_tipo.loc[c].values)


In [ ]:
plt.figure(figsize=(25,25))
sns.scatterplot(x="Latitude", y="Longitude",hue="Falha", data=db_igual)
plt.title("Ocorrências das cidades e tipos de falha de maiores frequências",fontdict= {'size':26})
plt.xlabel("Latitude", fontdict= {'size':26})
plt.ylabel("Longitude", fontdict= {'size':26})
plt.show()

In [ ]:
# Perda de Controle em voo, colisão com obstaculo durante decolagem e pouso, colisão com obstaculo no solo

In [ ]:
db_igual.insert(loc = 4, column = 'Cidade', value=range(len(igual)))
db_igual.insert(loc = 5, column = 'UF', value=range(len(igual)))

for i in igual:
    a = np.where(db_ocor.codigo_ocorrencia1 == i)
    b = np.where(db_igual.Codigo == i)
    db_igual.Cidade.loc[b] = db_ocor.ocorrencia_cidade.loc[a].values
    db_igual.UF.loc[b] = db_ocor.ocorrencia_uf.loc[a].values

In [ ]:
print('Frequencia acumulada de acordo com os Estados, em %')
a=(db_igual.UF.value_counts()/db_igual.UF.value_counts().sum())*100
print(a.cumsum())

In [ ]:
db_igual2 = db_igual.loc[db_igual.UF.isin(['SP','MG','PR','RJ'])]
df2 = pd.crosstab(db_igual2.Falha, db_igual2.UF)

plt.figure(1, figsize=(12,12))
df2.plot(kind = "barh", figsize=(12,12),stacked = True, title = "Falhas nos Estados de maior frequência")
plt.xlabel("Tipo de Falha")
plt.ylabel("Frequência")
plt.show()

# Análise Geral da Base Aeronaves

In [ ]:
db_aero.head()

In [ ]:
print('{0:.2f}% das ocorrências incluiram pelo menos 1 fatalidade.'.format(100-((db_aero.aeronave_fatalidades_total.value_counts()/db_aero.aeronave_fatalidades_total.value_counts().sum())*100)[0]))

In [ ]:
print(db_aero.aeronave_tipo_veiculo.value_counts())
print('Das {0} ocorrências com tipo de veículo especificado, houveram 9 tipos e {1:.2f}% foram envolvendo aviões.'.format((db_aero.aeronave_tipo_veiculo.shape[0]-81),(4268/5222)*100))

In [ ]:
db_aero2 = db_aero.loc[db_aero.aeronave_tipo_veiculo.isin(['AVIÃO'])]
print(db_aero2.aeronave_fabricante.value_counts())
print(db_aero2.aeronave_ano_fabricacao.value_counts())

In [ ]:
fig, ax1 = plt.subplots(figsize=(8,8))
ax2 = ax1.twinx()
ax1.bar(db_aero2.aeronave_fabricante.value_counts().index[:8],db_aero2.aeronave_fabricante.value_counts()[:8].values, color = 'gray')
ax2.plot(db_aero2.aeronave_fabricante.value_counts().index[:8],((db_aero2.aeronave_fabricante.value_counts()/db_aero2.aeronave_fabricante.value_counts().sum())*100).cumsum()[:8].values, color = 'b')
ax1.set_xlabel("Fabricante")
ax1.set_ylabel("Número de Ocorrências")
ax2.set_ylabel("Frequência Acumulada")
for tick in ax1.get_xticklabels():
    tick.set_rotation(55)
fig.align_labels() 
plt.title('Distribuição das Ocorrências pelo Fabricante do Avião')
plt.show()

print('Das ocorrências que envolveram aviões (80% do total) {0:.2f}% foram com apenas 8 Fabricantes (de um total de 113).'.format(((db_aero2.aeronave_fabricante.value_counts()/db_aero2.aeronave_fabricante.value_counts().sum())*100).cumsum().values[8]))

In [ ]:
fabri = db_aero2.aeronave_fabricante.value_counts().index[:8]
db_aero3 = db_aero2.loc[db_aero2.aeronave_fabricante.isin(fabri)]
#print((db_aero3.aeronave_ano_fabricacao.value_counts()/db_aero3.aeronave_ano_fabricacao.value_counts().sum()).cumsum())
db_aero3

In [ ]:
db_aero3.aeronave_motor_tipo.value_counts()

In [ ]:
db_aero3 = db_aero3.drop(db_aero3[db_aero3.aeronave_motor_tipo == '***'].index)
df3 = pd.crosstab(db_aero3.aeronave_fabricante, db_aero3.aeronave_motor_tipo)

print('Entre os tipos de falha, FALHA DO MOTOR EM VOO é o 3º mais frequente. Assim, o gráfico a seguir analisa o tipo do motor dentre os Fabricantes que obtiveram mais ocorrências')

plt.figure(1, figsize=(12,12))
df3.plot(kind = "barh", figsize=(12,12),stacked = True, title = "Tipo do motor pelos Fabricantes de maiores frequências")
plt.xlabel("Quantidade de Ocorrências")
plt.ylabel("Fabricante")
plt.show()

In [ ]:
db_igual.insert(loc = 6, column = 'Veiculo', value=range(len(igual)))
db_igual.insert(loc = 7, column = 'Fabricante', value=range(len(igual)))
db_igual.insert(loc = 8, column = 'ano_fabricacao', value=range(len(igual)))

for i in igual:
    a = np.where(db_ocor.codigo_ocorrencia1 == i)
    b = int(db_ocor.codigo_ocorrencia2.loc[a].values)
    c = np.where(db_aero.codigo_ocorrencia2 == b)
    d = np.where(db_igual.Codigo == i)
    if len(db_aero.aeronave_tipo_veiculo.loc[c].values)>1:
        db_igual.Veiculo.loc[d] = db_aero.aeronave_tipo_veiculo.loc[c].values[0]
        db_igual.Fabricante.loc[d] = db_aero.aeronave_fabricante.loc[c].values[0]
        db_igual.ano_fabricacao.loc[d] = db_aero.aeronave_ano_fabricacao.loc[c].values[0]
    else:        
        db_igual.Veiculo.loc[d] = db_aero.aeronave_tipo_veiculo.loc[c].values
        db_igual.Fabricante.loc[d] = db_aero.aeronave_fabricante.loc[c].values
        db_igual.ano_fabricacao.loc[d] = db_aero.aeronave_ano_fabricacao.loc[c].values

In [ ]:
print((db_igual.Veiculo.value_counts()/db_igual.Veiculo.value_counts().sum()).cumsum())
print((db_igual.Fabricante.value_counts()/db_igual.Fabricante.value_counts().sum()).cumsum())
print((db_igual.ano_fabricacao.value_counts()/db_igual.ano_fabricacao.value_counts().sum()).cumsum())

In [ ]:
print('Dentre as ocorrências sucedidas em 23% das cidades, 83.66% foram com aviões, as 8 maiores fabricantes detêm 66.64% dessas ocorrências.')


# Análise Geral da Base Fator Contribuinte

In [ ]:
db_contr.head()

In [ ]:
print(db_contr.fator_nome.value_counts())
print(db_contr.fator_aspecto.value_counts())
print(db_contr.fator_condicionante.value_counts())
print(db_contr.fator_area.value_counts())

In [ ]:
print('66.08% das ocorrências que tiveram o Fator classificado foram de Fator Operacional.')
print('Fator Humano com 33.30% das ocorrências classificadas')

In [ ]:
db_contr3 = db_contr.loc[db_contr.fator_area.isin(['FATOR OPERACIONAL'])]
print('Há {0} tipos de Fator Operacional contribuinte para as falhas.'.format(len(db_contr3.fator_nome.value_counts().index)))

In [ ]:
fig, ax1 = plt.subplots(figsize=(8,8))
ax2 = ax1.twinx()
ax1.bar(db_contr3.fator_nome.value_counts().index[:7],db_contr3.fator_nome.value_counts().values[:7], color = 'gray')
ax2.plot(db_contr3.fator_nome.value_counts().index[:7],((db_contr3.fator_nome.value_counts()/db_contr3.fator_nome.value_counts().sum())*100).cumsum()[:7].values, color = 'b')
ax1.set_ylabel("Quantidade de Ocorrências")
ax1.set_xlabel("Fator Contribuinte")
ax2.set_ylabel("Frequência Acumulada")
plt.title('Análise Fator Operacional')
for tick in ax1.get_xticklabels():
    tick.set_rotation(55)
fig.align_labels() 
plt.show()

In [ ]:
db_contr4 = db_contr.loc[db_contr.fator_area.isin(['FATOR HUMANO'])]
print('Dentro de Fator humano segue as 10 causas mais frequentes (%):')
print(((db_contr4.fator_nome.value_counts()/db_contr4.fator_nome.value_counts().sum())*100)[:10])

In [ ]:
db_igual.insert(loc = 9, column = 'fator_area', value=range(len(igual)))
db_igual.insert(loc = 10, column = 'fator_aspecto', value=range(len(igual)))
d=0
for i in igual:
    c = np.where(db_contr.codigo_ocorrencia3 == i)
    if len(db_contr.fator_area.loc[c].values)>1:
        db_igual.fator_area.loc[d] = db_contr.fator_area.loc[c].values[0]
        db_igual.fator_aspecto.loc[d] = db_contr.fator_aspecto.loc[c].values[0]
        d+=1
    else:
        db_igual.fator_area.loc[d] = db_contr.fator_area.loc[c].values
        db_igual.fator_aspecto.loc[d] = db_contr.fator_aspecto.loc[c].values
        d+=1

In [ ]:
print('Dentre as ocorrências das 146 cidades já selecionadas, temos:\n',db_igual.fator_area.value_counts()[:6])

# Análise Geral da Base Recomendação

In [ ]:
db_reco.head()

In [ ]:
print(db_reco.recomendacao_status.value_counts()/db_reco.recomendacao_status.value_counts().sum())
print(db_reco.recomendacao_destinatario_sigla.value_counts()/db_reco.recomendacao_destinatario_sigla.value_counts().sum())
print('Do total de {0} recomendações realizadas, 60.1% foram cumpridas e 7.17% de forma alternativa.'.format(db_reco.shape[0]))
print('Em relação ao orgão de destino, 81.72% foram para a ANAC.')

In [ ]:
db_reco2 = db_reco.loc[db_reco.recomendacao_status.isin(['CUMPRIDA','CUMPRIDA DE FORMA ALTERNATIVA'])]
print(db_reco2.recomendacao_destinatario_sigla.value_counts()/db_reco2.recomendacao_destinatario_sigla.value_counts().sum())

In [ ]:
db_reco2.insert(loc = 9, column = 'ocorrencia_dia', value=range(len(db_reco2.recomendacao_status)))
db_reco2.insert(loc = 10, column = 'ocorrencia_tipo', value=range(len(db_reco2.recomendacao_status)))

for i in db_reco2.index:
    c = np.where(db_ocor.codigo_ocorrencia4 == db_reco2.codigo_ocorrencia4[i])
    db_reco2.ocorrencia_dia.loc[i] = db_ocor.ocorrencia_dia.loc[c].values
    db_reco2.ocorrencia_tipo.loc[i] = db_ocor_tp.ocorrencia_tipo.loc[c].values
    

In [ ]:
print('Tipos de falha entre as recomendações Cumpridas:\n',db_reco2.ocorrencia_tipo.value_counts()/db_reco2.ocorrencia_tipo.value_counts().sum())

In [ ]:
db_reco2.ocorrencia_dia[3]

In [ ]:
db_reco3 = db_reco.loc[db_reco.recomendacao_status.isin(['NÃO CUMPRIDA'])]
print('As recomendações foram direcionadas a 83 orgãos diferentes, {0} não foram cumpridas somente pela ANAC e DECEA'.format(db_reco3.shape[0]))
print(db_reco3.recomendacao_destinatario_sigla.value_counts()/db_reco3.recomendacao_destinatario_sigla.value_counts().sum())

In [ ]:
db_reco3.insert(loc = 9, column = 'ocorrencia_dia', value=range(len(db_reco3.recomendacao_status)))
db_reco3.insert(loc = 10, column = 'ocorrencia_tipo', value=range(len(db_reco3.recomendacao_status)))

for i in db_reco3.index:
    c = np.where(db_ocor.codigo_ocorrencia4 == db_reco3.codigo_ocorrencia4[i])
    db_reco3.ocorrencia_dia.loc[i] = db_ocor.ocorrencia_dia.loc[c].values
    db_reco3.ocorrencia_tipo.loc[i] = db_ocor_tp.ocorrencia_tipo.loc[c].values
    

In [ ]:
db_reco3.ocorrencia_tipo = db_reco3.ocorrencia_tipo.astype("str")
df4 = pd.crosstab(db_reco3.ocorrencia_tipo, db_reco3.recomendacao_destinatario_sigla)
df4.sort_values('ANAC',inplace=True)
plt.figure(1, figsize=(12,12))
df4.plot(kind = "barh", figsize=(12,12),stacked = True, title = "Tipos de Falhas com recomendações não cumpridas")
plt.xlabel("Quantidade de Ocorrências")
plt.ylabel("Tipo de falha")
plt.show()

In [ ]:
db_igual.insert(loc = 11, column = 'reco_status', value=range(len(igual)))
db_igual.insert(loc = 12, column = 'reco_dest_sigla', value=range(len(igual)))
db_igual.insert(loc = 13, column = 'reco_cont', value=range(len(igual)))
d=0
for i in igual:
    c = np.where(db_reco.codigo_ocorrencia4 == i)
    if len(db_contr.fator_area.loc[c].values)>1:
        db_igual.reco_status.loc[d] = db_reco.recomendacao_status.loc[c].values[0]
        db_igual.reco_dest_sigla.loc[d] = db_reco.recomendacao_destinatario_sigla.loc[c].values[0]
        db_igual.reco_cont.loc[d] = db_reco.recomendacao_conteudo.loc[c].values[0]
        d+=1
    else:
        db_igual.reco_status.loc[d] = db_reco.recomendacao_status.loc[c].values
        db_igual.reco_dest_sigla.loc[d] = db_reco.recomendacao_destinatario_sigla.loc[c].values
        db_igual.reco_cont.loc[d] = db_reco.recomendacao_conteudo.loc[c].values
        d+=1

In [ ]:
print('Dentre as ocorrências das 146 cidades:\n',db_igual.reco_status.value_counts()[:8]/db_igual.reco_status.value_counts()[:8].sum())

In [ ]:
db_igual.reco_status = db_igual.reco_status.astype('str')
db_igual1 = db_igual.loc[db_igual.reco_status.isin(['CUMPRIDA','[CUMPRIDA]','CUMPRIDA DE FORMA ALTERNATIVA','[CUMPRIDA DE FORMA ALTERNATIVA]'])]
print('Tipo de falha das recomendações cumpridas:\n',db_igual1.Falha.value_counts())

In [ ]:
db_igual1b = db_igual1.loc[db_igual1.UF.isin(['SP','RS','MT','PA'])]
print('Frequência dos estados com recomendações cumpridas dentre as 246 ciades:\n',db_igual1.UF.value_counts()/db_igual1.UF.value_counts().sum())
print('Tipo de falha das recomendações cumpridas para 4 estados:\n',db_igual1b.Falha.value_counts())

In [ ]:
df5 = pd.crosstab(db_igual1b.Falha, db_igual1b.UF)
df5.sort_values('SP',inplace=True)
plt.figure(1, figsize=(8,8))
df5.plot(kind = "barh", figsize=(12,12),stacked = True, title = "Tipos de Falhas com recomendações cumpridas")
plt.xlabel("Quantidade de Ocorrências")
plt.ylabel("Tipo de falha")
plt.show()

In [ ]:
db_igual2 = db_igual.loc[db_igual.reco_status.isin(['NÃO CUMPRIDA','[NÃO CUMPRIDA]'])]
print('Tipo de falha das recomendações não cumpridas:\n',db_igual2.Falha.value_counts())

In [ ]:
print('Cidades com recomendações não cumpridas:\n',db_igual2.Cidade.value_counts())

In [ ]:
print(db_igual2.Falha.loc[db_igual2.Cidade.isin(['SÃO PAULO'])])
print('Foram em diferentes ocorrências e tipos de falhas.')